In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import k3d
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sys
import tqdm

In [3]:
import sys

In [4]:
sys.path.append('/code/challenges/cvpr2020/abc_sharpness_fields_extraction/scoring_program/')

In [5]:
import utilities as u

In [6]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768, 
                      cmap_distances=k3d.colormaps.basic_color_maps.WarmCool,
                      distance_range=None):
    
    plot = k3d.plot(height=plot_height, camera_fov=30.0)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            if None is distance_range:
                distance_range = [0, np.max(samples_distances)]

            colors = k3d.helpers.map_colors(
                samples_distances, cmap_distances, distance_range
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            directions_to_plot = np.hstack((samples, samples + directions))
            
            for i, dir_to_plot in enumerate(directions_to_plot):
                dir_to_plot = dir_to_plot.reshape((2, 3))
                if np.all(dir_to_plot[0] == dir_to_plot[1]):
                    continue
                color = int(colors[i]) if None is not colors else samples_color
                plt_line = k3d.line(dir_to_plot, 
                                    shader='mesh', width=directions_width, color=color)
                plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.camera_auto_fit = True
    plot.display()
    
    return plot

In [7]:
!ls /logs/alberts_dgcnn_predictions/

test  val


In [30]:
# data_field, target_field = u.read_targz_file('/data/abc_sharpness_fields_extraction/npy/high_res_train_0_data.tar.gz')
data_field, _ = u.read_targz_file('/data/abc_sharpness_fields_extraction/npy/low_res_train_0_data.tar.gz', read_distances=False)
_, target_field = u.read_targz_file('/data/abc_sharpness_fields_extraction/npy/low_res_train_0_target.tar.gz', read_points=False)


In [32]:
data_field.shape, target_field.shape

((16384, 4096, 3), (16384, 4096, 0))

In [29]:

idx = np.random.choice(16000)
print(idx)
# idx = 1756

# data, target = u.read_txt_file('/data/txt/low_res/train_0/field_{0:06d}.txt'.format(idx))

# data_field, _ = u.read_txt_file('/data/txt/low_res/val_0_data/field_{0:06d}_data.txt'.format(idx), read_distances=False)
# _, target = u.read_txt_file('/data/txt/low_res/val_0_target/field_{0:06d}_target.txt'.format(idx), read_points=False)
# data, _ = u.read_targz_file('/data/npy/low_res_val_0_data.tar.gz', read_distances=False)
# _, target = u.read_targz_file('/data/npy/low_res_val_0_target.tar.gz', read_points=False)

display_sharpness(
    samples=data_field[idx],
    samples_distances=target_field[idx],
    samples_psize=0.125
)

686


Output()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[4.5, 4.5, 4.5, 0.0…

# Shape Segmentation

In [634]:
import os
def _p(filename):
    return os.path.join('/data/abc_geometric_shape_segmentation/', filename)

In [635]:
data_seg, _ = u.read_targz_file(_p('npy/high_res_val_0_data.tar.gz'), read_distances=False)
_, target_seg_gt = u.read_targz_file(_p('npy/high_res_val_0_target.tar.gz'), read_points=False)

# data_seg, _ = u.read_targz_file('/data/abc_geometric_shape_segmentation/npy/high_res_val_0_data.tar.gz', read_distances=False)
# _, target_seg = u.read_targz_file('/data/abc_geometric_shape_segmentation/npy/high_res_val_0_target.tar.gz', read_points=False)

In [718]:
idx = 12155
_, target_seg_pred = u.read_txt_file(
    '/logs/alberts_dgcnn_predictions/val/voronoi/high_res/segmentation_{0:06d}_target.txt'.format(idx), 
    read_points=False)

In [719]:
target_seg_gt[idx].sum(), target_seg_pred.sum()

(0.0, 0.0)

In [717]:
res = 'high'
res_number = {'high': 0.02, 'med': 0.05, 'low': 0.125}[res]

# idx = np.random.choice(16000)
# print(idx)
# idx = 1756

# data_seg, target_seg = u.read_txt_file(
#     '/data/abc_geometric_shape_segmentation/txt/low_res/train_0/segmentation_{0:06d}.txt'.format(idx))

# data_seg, _ = u.read_txt_file(
#     '/data/abc_geometric_shape_segmentation/txt/{0}_res/test_0_data/segmentation_{1:06d}_data.txt'.format(res, idx), read_distances=False)
# _, target_seg = u.read_txt_file(
#     '/data/abc_geometric_shape_segmentation/txt/{0}_res/test_0_target/segmentation_{1:06d}_target.txt'.format(res, idx), read_points=False)

# data_seg, _ = u.read_targz_file('/data/npy/low_res_val_0_data.tar.gz', read_distances=False)
# _, target_seg = u.read_targz_file('/data/npy/low_res_val_0_target.tar.gz', read_points=False)


_= display_sharpness(
    samples=data_seg[idx],
#     samples_distances=target_seg_gt[idx],
    samples_distances=target_seg_pred,
    samples_psize=res_number,
    cmap_distances=k3d.colormaps.basic_color_maps.CoolWarm
)

Output()